In [1]:
from datasets import load_dataset
import numpy as np

C:\Users\sthor\anaconda3\envs\NLP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# https://huggingface.co/datasets/amazon_polarity
# takes a long time to process, you may want to try it yourself
# dataset = load_dataset("amazon_polarity")

In [3]:
raw_datasets = load_dataset("glue", "sst2")

Dataset glue downloaded and prepared to C:/Users/sthor/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<?, ?it/s]


In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [5]:
raw_datasets['train']

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})

In [6]:
dir(raw_datasets['train'])

['_TF_DATASET_REFS',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getitems__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_local_temp_path',
 '_check_index_is_initialized',
 '_data',
 '_estimate_nbytes',
 '_fingerprint',
 '_format_columns',
 '_format_kwargs',
 '_format_type',
 '_generate_examples_from_shards',
 '_get_cache_file_path',
 '_get_output_signature',
 '_getitem',
 '_indexes',
 '_indices',
 '_info',
 '_map_single',
 '_new_dataset_with_indices',
 '_output_all_columns',
 '_push_parquet_shards_to_hub',
 '_save_to_disk_single',
 '_select_contiguous',
 '_select_with_indices_mapping',
 '_split',
 'add_column',
 'add

In [7]:
type(raw_datasets['train'])

datasets.arrow_dataset.Dataset

In [8]:
raw_datasets['train'].data

MemoryMappedTable
sentence: string
label: int64
idx: int32
----
sentence: [["hide new secretions from the parental units ","contains no wit , only labored gags ","that loves its characters and communicates something rather beautiful about human nature ","remains utterly satisfied to remain the same throughout ","on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ",...,"you wish you were at home watching that movie instead of in the theater watching this one ","'s no point in extracting the bare bones of byatt 's plot for purposes of bland hollywood romance ","underdeveloped ","the jokes are flat ","a heartening tale of small victories "],["suspense , intriguing characters and bizarre bank robberies , ","a gritty police thriller with all the dysfunctional family dynamics one could wish for ","with a wonderful ensemble cast of characters that bring the routine day to day struggles of the working class to life ","nonetheless appreciates the art and reveals a music sc

In [9]:
raw_datasets['train'][0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [10]:
raw_datasets['train'][50000:50003]

{'sentence': ['glow ',
  'a classical dramatic animated feature ',
  'best espionage picture '],
 'label': [1, 1, 1],
 'idx': [50000, 50001, 50002]}

In [11]:
raw_datasets['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [12]:
from transformers import AutoTokenizer

In [13]:
# checkpoint = "bert-base-uncased"
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

C:\Users\sthor\anaconda3\envs\NLP\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sthor\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:
tokenized_sentences = tokenizer(raw_datasets['train'][0:3]['sentence'])
from pprint import pprint
pprint(tokenized_sentences)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102],
               [101,
                3397,
                2053,
                15966,
                1010,
                2069,
                4450,
                2098,
                18201,
                2015,
                102],
               [101,
                2008,
                7459,
                2049,
                3494,
                1998,
                10639,
                2015,
                2242,
                2738,
                3376,
                2055,
                2529,
                3267,
                102]]}


In [15]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence'], truncation=True)

In [16]:
tokenized_datasets = raw_datasets.map(tokenize_fn, batched=True)

In [17]:
from transformers import TrainingArguments

In [18]:
training_args = TrainingArguments(
  'my_trainer',
  evaluation_strategy='epoch',
  save_strategy='epoch',
  num_train_epochs=1,
)

In [19]:
from transformers import AutoModelForSequenceClassification

In [20]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [21]:
type(model)

transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification

In [22]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [23]:
# !pip install torchinfo

In [24]:
from torchinfo import summary
# summary(model, input_size=(16,512), dtypes=['torch.IntTensor'], device='cpu')
summary(model)

Layer (type:depth-idx)                                  Param #
DistilBertForSequenceClassification                     --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             42,527,232
├─Linear: 1-2                                           590,592
├─Linear: 1-3                                           1,538
├─Dropout: 1-4                                          --
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0

In [25]:
params_before = []
for name, p in model.named_parameters():
  params_before.append(p.detach().cpu().numpy())

In [27]:
params_before[0]

array([[-0.01664949, -0.06661227, -0.01632868, ..., -0.01999032,
        -0.05139988, -0.0263568 ],
       [-0.01319846, -0.06733431, -0.01605646, ..., -0.0226614 ,
        -0.05537301, -0.02600443],
       [-0.01759106, -0.07094341, -0.01443494, ..., -0.02457913,
        -0.05956192, -0.0231829 ],
       ...,
       [-0.0231029 , -0.05878259, -0.01048967, ..., -0.01945743,
        -0.02615411, -0.02118432],
       [-0.0490171 , -0.05614787, -0.00465348, ..., -0.01065376,
        -0.01797333, -0.02187675],
       [-0.00646111, -0.0914881 , -0.00254872, ..., -0.01505679,
        -0.05040044,  0.04597744]], dtype=float32)

In [28]:
from transformers import Trainer

In [29]:
from datasets import load_metric

In [30]:
metric = load_metric("glue", "sst2")

C:\Users\sthor\AppData\Local\Temp\ipykernel_28000\4041007664.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "sst2")


In [31]:
metric.compute(predictions=[1, 0, 1], references=[1, 0, 0])

{'accuracy': 0.6666666666666666}

In [32]:
def compute_metrics(logits_and_labels):
  # metric = load_metric("glue", "sst2")
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [33]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [34]:
trainer.train()

C:\Users\sthor\anaconda3\envs\NLP\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.199300,0.326016,0.913991


TrainOutput(global_step=8419, training_loss=0.26445640222323114, metrics={'train_runtime': 330.038, 'train_samples_per_second': 204.064, 'train_steps_per_second': 25.509, 'total_flos': 518400815624736.0, 'train_loss': 0.26445640222323114, 'epoch': 1.0})

In [35]:
trainer.save_model('my_saved_model')

In [37]:
!dir my_saved_model

 El volumen de la unidad C es Windows
 El número de serie del volumen es: 14FB-F245

 Directorio de C:\Users\sthor\Programas_codigo_fuente\Udemy\Data Science - Transformers for Natural Language Processing\Fine-Tuning (Intermediate)\my_saved_model

01/06/2023  15:46    <DIR>          .
01/06/2023  15:46    <DIR>          ..
01/06/2023  15:46               640 config.json
01/06/2023  15:46       267,855,533 pytorch_model.bin
01/06/2023  15:46               132 special_tokens_map.json
01/06/2023  15:46           711,494 tokenizer.json
01/06/2023  15:46               328 tokenizer_config.json
01/06/2023  15:46             3,515 training_args.bin
01/06/2023  15:46           231,508 vocab.txt
               7 archivos    268,803,150 bytes
               2 dirs  1,426,254,323,712 bytes libres


In [38]:
from transformers import pipeline

In [39]:
newmodel = pipeline('text-classification', model='my_saved_model', device=0)

In [40]:
newmodel('This movie is great!')

[{'label': 'LABEL_1', 'score': 0.9996622800827026}]

In [41]:
newmodel('This movie sucks')

[{'label': 'LABEL_0', 'score': 0.9938357472419739}]

In [47]:
!type ".\my_saved_model\config.json"

{
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.27.3",
  "vocab_size": 30522
}


In [48]:
import json

In [49]:
config_path = 'my_saved_model/config.json'
with open(config_path) as f:
  j = json.load(f)

j['id2label'] = {0: 'negative', 1: 'positive'}

with open(config_path, 'w') as f:
  json.dump(j, f, indent=2)

In [50]:
!type ".\my_saved_model\config.json"

{
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.27.3",
  "vocab_size": 30522,
  "id2label": {
    "0": "negative",
    "1": "positive"
  }
}


In [51]:
newmodel = pipeline('text-classification', model='my_saved_model', device=0)

In [52]:
newmodel('This movie is great!')

[{'label': 'positive', 'score': 0.9996622800827026}]

In [53]:
params_after = []
for name, p in model.named_parameters():
  params_after.append(p.detach().cpu().numpy())

In [54]:
for p1, p2 in zip(params_before, params_after):
  print(np.sum(np.abs(p1 - p2)))

13424.296
91.567604
1.7471666
1.1312364
1303.0519
1.7988651
1296.923
0.0026480197
1195.1136
1.0632279
1124.1174
0.8485209
1.6666763
0.8466856
4896.44
5.6765833
4521.9736
0.75138915
1.5470902
0.75240415
1275.5842
1.4804773
1285.733
0.002688453
1117.4008
0.8734672
1063.9297
0.7558677
1.5182885
0.7520532
4839.701
5.312435
4446.66
0.7036563
1.4291208
0.75127447
1270.0276
1.652457
1278.4675
0.0025629285
1119.5691
0.809929
1096.4158
0.73151714
1.5167137
0.77988786
4868.317
5.530164
4336.0747
0.6889404
1.4033606
0.62280065
1260.224
1.4314666
1267.6895
0.0026685758
1124.3252
0.74784315
1071.2489
0.73180175
1.4313605
0.7386643
4763.301
5.448554
4117.5005
0.7367306
1.3270562
0.6433481
1177.4824
1.5391057
1183.0364
0.0018984021
967.2154
0.73319227
976.0718
0.93023396
1.3540028
0.96040344
4293.0225
5.1459217
3379.285
0.77332926
1.3298125
0.7713404
1093.6649
1.3275683
1090.3699
0.0010320422
899.7998
0.9288232
899.84454
1.0671304
1.3111801
1.2423174
3577.5112
4.58383
3093.057
0.90285826
1.2532837
0.